Context: This script executes the final data integration phase (06_final_merge.ipynb). It bridges the gap between the two primary data streams of the project: the Micro-Level Data (County Socio-Economics and Voting Results) and the Macro-Level Data (National Candidate Rhetoric).

In [22]:
import pandas as pd
from pathlib import Path
# ==========================================
# STEP 1: LOAD NLP SENTIMENT DATA
# ==========================================
# Load the detailed sentiment analysis dataset generated in the previous notebook (05_nlp.ipynb).
path_nlp = PROCESSED_DIR / "political_sentiment_DETAILED.csv"
df_nlp = pd.read_csv(path_nlp)
print("✅ NLP Sentiment Table successfully loaded.")

# ==========================================
# STEP 2: LOAD MASTER SOCIO-ECONOMIC DATA
# ==========================================
# Load the aggregated dataset containing county-level election results and socio-economic indicators.
current_dir = Path.cwd()
PROJECT_ROOT = current_dir.parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
FIG_DIR = PROJECT_ROOT / 'figures'

path_master = PROCESSED_DIR / "master_table_elections.csv"
output_path = PROCESSED_DIR/ "PROJET_USA_FINAL_DATASET.csv"


try:
    df_master = pd.read_csv(path_master)
    print(f"✅ Master Table loaded. Dimensions: {df_master.shape}")
except FileNotFoundError:
    print("❌ ERROR: File not found.")
    print("Please verify the file path.")
    raise 

# ==========================================
# STEP 3: NLP DATA TRANSFORMATION (PIVOTING)
# ==========================================
# The objective is to merge national-level sentiment scores (Party-specific) 
# into the county-level dataset. We must separate Democratic and Republican scores 
# into distinct columns to allow for a horizontal merge.

# A. Isolate and Rename Democratic Metrics
df_dem = df_nlp[df_nlp['party'] == 'Democrat'].copy()
# Add prefix 'DEM_' to all sentiment columns
cols_to_rename = {col: f"DEM_{col}" for col in df_dem.columns if col not in ['year']}
df_dem = df_dem.rename(columns=cols_to_rename).drop(columns=['DEM_party'])

# B. Isolate and Rename Republican Metrics
df_rep = df_nlp[df_nlp['party'] == 'Republican'].copy()
# Add prefix 'REP_' to all sentiment columns
cols_to_rename = {col: f"REP_{col}" for col in df_rep.columns if col not in ['year']}
df_rep = df_rep.rename(columns=cols_to_rename).drop(columns=['REP_party'])

# ==========================================
# STEP 4: DATA INTEGRATION (MERGE)
# ==========================================
print("Initiating data merge...")

# Schema Harmonization: Ensure the joining key is named 'year' in the master table.
if 'Year' in df_master.columns:
    df_master = df_master.rename(columns={'Year': 'year'})

# Perform a Left Join on 'year'.
# This operation broadcasts the national-level sentiment scores to every county observation 
# for the corresponding election cycle.
df_final = pd.merge(df_master, df_dem, on='year', how='left')
df_final = pd.merge(df_final, df_rep, on='year', how='left')

# ==========================================
# STEP 5: FEATURE ENGINEERING (INTERACTIONS)
# ==========================================
# Calculate the Sentiment Differential.
# Positive values (>0) indicate the Democratic candidate expressed more positive sentiment 
# than the Republican candidate for that year.
df_final['SENTIMENT_DIFF'] = df_final['DEM_sentiment_mean'] - df_final['REP_sentiment_mean']

# ==========================================
# STEP 6: FINAL EXPORT
# ==========================================

df_final.to_csv(output_path, index=False)

print(f"\n🚀 PROCESS COMPLETE. The final dataset is available at:")
print(output_path)
print("Preview of integrated NLP columns:")
print(df_final[['year', 'DEM_sentiment_mean', 'REP_sentiment_mean']].head())

✅ NLP Sentiment Table successfully loaded.
✅ Master Table loaded. Dimensions: (21742, 15)
Initiating data merge...

🚀 PROCESS COMPLETE. The final dataset is available at:
/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/processed/PROJET_USA_FINAL_DATASET.csv
Preview of integrated NLP columns:
   year  DEM_sentiment_mean  REP_sentiment_mean
0  2000            0.146626            0.131232
1  2000            0.146626            0.131232
2  2000            0.146626            0.131232
3  2000            0.146626            0.131232
4  2000            0.146626            0.131232


This step integrates the national-level NLP sentiment indicators into the county-level master dataset to create a unified modeling table. It loads the sentiment panel, splits it into Democratic and Republican blocks with distinct column prefixes, merges both onto the master dataset by election year (broadcasting the yearly sentiment scores to all counties), engineers a sentiment differential feature, and exports the resulting final dataset as a single CSV ready for joint socio-economic and NLP-based predictive modeling.

Context: This final segment of the 06_final_merge.ipynb notebook performs a Quality Assurance (QA) check on the merged dataset (df_final). It ensures that the integration of the macro-level sentiment data with the micro-level county data was executed correctly before the file is finalized for the final report.

In [18]:
# ==========================================
# STEP 7: FINAL DATASET VERIFICATION
# ==========================================

# 1. Variable Inspection
# Output the complete list of column names to verify the final schema 
# and ensure no redundant columns (e.g., '_x', '_y') remain.
print("--- FULL VARIABLE LIST ---")
print(df_final.columns.tolist())

# 2. Sample Observation Inspection
# Display the first row to verify the successful integration of socio-economic and NLP data.
print("\n--- SAMPLE OBSERVATION (HEAD) ---")

# Define a subset of key columns to inspect:
# - Temporal Identifier: 'year'
# - Geographic Identifier: 'county_name' (if available in master table)
# - NLP Features: 'DEM_sentiment_mean', 'REP_sentiment_mean', 'SENTIMENT_DIFF'
cols_to_check = ['year', 'county_name', 'DEM_sentiment_mean', 'REP_sentiment_mean', 'SENTIMENT_DIFF']

# Robust selection: Only include columns that actually exist in the final dataframe
# to prevent KeyErrors if the master table schema varies slightly.
available_cols = [c for c in cols_to_check if c in df_final.columns]

# Print the values for the selected columns
print(df_final[available_cols].head(1))

--- FULL VARIABLE LIST ---
['year', 'state_po', 'county_name', 'fips', 'DEMOCRAT', 'REPUBLICAN', 'unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct', 'education_bachelors_pct', 'median_age', 'white_pct', 'black_pct', 'hispanic_pct', 'DEM_sentiment_platform', 'DEM_sentiment_speech', 'DEM_subjectivity_platform', 'DEM_subjectivity_speech', 'DEM_sentiment_mean', 'DEM_subjectivity_mean', 'REP_sentiment_platform', 'REP_sentiment_speech', 'REP_subjectivity_platform', 'REP_subjectivity_speech', 'REP_sentiment_mean', 'REP_subjectivity_mean', 'SENTIMENT_DIFF']

--- SAMPLE OBSERVATION (HEAD) ---
   year  county_name  DEM_sentiment_mean  REP_sentiment_mean  SENTIMENT_DIFF
0  2000  DISTRICT 13            0.146626            0.131232        0.015394


This final verification step checks the integrity of the integrated dataset by inspecting the full list of variables and confirming that no merge artifacts remain. It then displays a sample observation to validate that socio-economic indicators and NLP-derived sentiment features have been correctly combined, ensuring the dataset is structurally sound and ready for final analysis or modeling.

In [19]:
print(df_final[df_final['year'] == 2020][['year', 'DEM_sentiment_mean', 'REP_sentiment_mean', 'SENTIMENT_DIFF']])

       year  DEM_sentiment_mean  REP_sentiment_mean  SENTIMENT_DIFF
15530  2020            0.129088            0.157135       -0.028047
15531  2020            0.129088            0.157135       -0.028047
15532  2020            0.129088            0.157135       -0.028047
15533  2020            0.129088            0.157135       -0.028047
15534  2020            0.129088            0.157135       -0.028047
...     ...                 ...                 ...             ...
18632  2020            0.129088            0.157135       -0.028047
18633  2020            0.129088            0.157135       -0.028047
18634  2020            0.129088            0.157135       -0.028047
18635  2020            0.129088            0.157135       -0.028047
18636  2020            0.129088            0.157135       -0.028047

[3107 rows x 4 columns]


Methodological Note: Interpretation of Macro-Level Sentiment Variables

Is it expected that sentiment scores are identical across all counties for a given year? Yes. This uniformity is methodologically sound and consistent with the research design.

This concept is critical for understanding the econometric specification of the model.

1. Structural Distinction: Temporal vs. Spatial Variables

The uniformity arises because the dataset combines variables from two different levels of analysis:

Sentiment Variables are National (Temporal): Presidential candidate speeches and party manifestos are "national public goods." When a candidate releases a platform in 2020, that text is identical whether the voter resides in Texas, New York, or California. There is no unique "Democratic Platform" specifically tailored for Miami-Dade County.

Socio-Economic Variables are Local (Spatial): Unemployment rates or demographics vary from county to county.

Technical Implication: During the merging process on the year key, the single national sentiment value (e.g., -0.02) is technically "broadcast" to all ~3,000 county observations for that specific year.

2. The Identification Strategy: Longitudinal Variation

Since there is no cross-sectional variation in sentiment within a single year (Variance = 0), the model does not compare County A against County B based on sentiment.

Instead, the model relies on Longitudinal (Time-Series) Variation. It analyzes how changes in national rhetoric over time correlate with shifts in local voting patterns.

Example: "In 2016, the national sentiment was positive (+0.05). In 2020, it became negative (-0.02). Did this aggregate shift correlate with a corresponding fluctuation in vote shares?"

This approach measures the impact of a national rhetorical shock on local electoral outcomes.

3. Advanced Econometric Strategy: Interaction Terms

To introduce local variation into the sentiment analysis—a powerful econometric technique, we must construct Interaction Terms.

This addresses the hypothesis of heterogeneous effects: Does a positive speech affect a wealthy county differently than a distressed county?

Implementation: You create a new variable X 
new
​	
 =Sentiment×Income.

Interpretation: This allows the model to determine if the marginal effect of candidate optimism is conditional on the local economic context.

Context: This final code block in 06_final_merge.ipynb implements the "Pro" tip discussed regarding Heterogeneous Treatment Effects. By constructing Interaction Terms, the dataset now supports advanced econometric modeling that goes beyond simple additive effects.

In [20]:
# ==========================================
# STEP 7: CREATION OF INTERACTION TERMS (ECONOMETRIC STRATEGY)
# ==========================================

# 1. Unemployment Interaction
# Hypothesis: The impact of national political rhetoric is conditional on the local economic distress.
# A positive speech might have a diminished (or negative) effect in counties with high unemployment.
col_unemployment = 'unemployment_rate' 

# Construct the interaction term: X_new = (Sentiment Differential) * (Local Unemployment Rate)
# Note: 'INTERACT_Sentiment_Chomage' renamed to 'INTERACT_Sentiment_Unemployment' for consistency.
df_final['INTERACT_Sentiment_Unemployment'] = df_final['SENTIMENT_DIFF'] * df_final[col_unemployment]

# 2. Income Interaction (Optional)
# Hypothesis: Does optimistic rhetoric resonate more effectively in wealthier counties?
# We verify the existence of 'median_income' (defined in 02_cleaning.ipynb) before calculating.
if 'median_income' in df_final.columns:
    df_final['INTERACT_Sentiment_Income'] = df_final['SENTIMENT_DIFF'] * df_final['median_income']

# ==========================================
# STEP 8: VERIFICATION
# ==========================================
print("Interaction terms successfully generated:")
# Inspect the new columns to ensure the multiplication logic applied correctly.
print(df_final[['year', 'SENTIMENT_DIFF', col_unemployment, 'INTERACT_Sentiment_Unemployment']].head())

# ==========================================
# STEP 9: FINAL OVERWRITE
# ==========================================
# Overwrite the final dataset to include these critical econometric variables.
df_final.to_csv(output_path, index=False)
print("✅ Final dataset updated and saved with interaction terms.")

Interaction terms successfully generated:
   year  SENTIMENT_DIFF  unemployment_rate  INTERACT_Sentiment_Unemployment
0  2000        0.015394                3.8                         0.058499
1  2000        0.015394                4.1                         0.063117
2  2000        0.015394                4.6                         0.070814
3  2000        0.015394                2.5                         0.038486
4  2000        0.015394                3.2                         0.049262


✅ Final dataset updated and saved with interaction terms.


This step extends the dataset with interaction terms to support an econometric analysis of conditional effects. By interacting the national sentiment differential with local unemployment (and, optionally, income), it allows the impact of political rhetoric to vary with local economic conditions, and the final dataset is updated to include these variables for subsequent regression-based analysis.

This step extends the dataset with interaction terms to support an econometric analysis of conditional effects. By interacting the national sentiment differential with local unemployment (and, optionally, income), it allows the impact of political rhetoric to vary with local economic conditions, and the final dataset is updated to include these variables for subsequent regression-based analysis.

In [21]:
print(df_final.columns.tolist())

['year', 'state_po', 'county_name', 'fips', 'DEMOCRAT', 'REPUBLICAN', 'unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct', 'education_bachelors_pct', 'median_age', 'white_pct', 'black_pct', 'hispanic_pct', 'DEM_sentiment_platform', 'DEM_sentiment_speech', 'DEM_subjectivity_platform', 'DEM_subjectivity_speech', 'DEM_sentiment_mean', 'DEM_subjectivity_mean', 'REP_sentiment_platform', 'REP_sentiment_speech', 'REP_subjectivity_platform', 'REP_subjectivity_speech', 'REP_sentiment_mean', 'REP_subjectivity_mean', 'SENTIMENT_DIFF', 'INTERACT_Sentiment_Unemployment', 'INTERACT_Sentiment_Income']


Econometric Interpretation of the Interaction Coefficient

When analyzing the regression results, the coefficient (β) associated with the interaction term (INTERACT_Sentiment_Unemployment) essentially tests for Heterogeneous Treatment Effects. It indicates whether the impact of the independent variable (Sentiment) changes depending on the value of the moderator variable (Unemployment).

1. If the Coefficient is POSITIVE (β>0): Amplification Effect

Interpretation: This indicates that the effect of the sentiment differential is magnified by local unemployment.

Context: It suggests a synergistic relationship where positive political rhetoric resonates more effectively in economically distressed areas. In this scenario, high unemployment acts as a catalyst, making the electorate more receptive to optimistic messaging.

2. If the Coefficient is NEGATIVE (β<0): Attenuation Effect

Interpretation: This indicates that the effect of the sentiment differential is dampened or diminished as the unemployment rate increases.

Context: It suggests that in regions characterized by high unemployment, the influence of national political rhetoric is weakened. Empirically, this implies that local economic realities override rhetorical appeals; essentially, when economic hardship is severe, voters are less swayed by the "tone" of the campaign, rendering positive sentiment less effective than in prosperous areas.